In [3]:
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd

In [4]:
def A(pN,pM, Xi,m):
    p1 = pN-Xi*(pM-pN)
    pm = pM+Xi*(pM-pN)
    A = [p1]
    for i in range (1,m):
        new = p1 + (i-1)*(pm-p1)/(m-1)
        A.append(new)
    return A

In [5]:
A = A(1.78,2,0.1,15)

In [6]:
len(A)

15

In [9]:
def S(A): 
    S = []
    for i in range (len(A)):
        for j in range (len(A)): 
            S.append([A[i],A[j]])
    return S
    

In [10]:
S = S(A)

In [11]:
s_ex = S[100]

In [12]:
def pi(s): 
    return (s[0]-1)*np.exp(4*(2-s[0]))/(np.exp(4*(2-s[0]))+np.exp(4*(2-s[1]))-1)

In [13]:
pi(s_ex)

0.7188056694679223

In [29]:
def env(A,pi):
    env = np.zeros((len(A),len(A)))
    for i in range (len(A)):
        for j in range (len(A)):
            env[i,j] = pi([A[i],A[j]])
    return env       

In [30]:
env(A,pi)

array([[0.46785559, 0.46785559, 0.48982103, 0.51211767, 0.53468835,
        0.55747289, 0.58040877, 0.60343175, 0.62647662, 0.64947792,
        0.67237067, 0.69509113, 0.71757748, 0.73977048, 0.76161406],
       [0.46785559, 0.46785559, 0.48982103, 0.51211767, 0.53468835,
        0.55747289, 0.58040877, 0.60343175, 0.62647662, 0.64947792,
        0.67237067, 0.69509113, 0.71757748, 0.73977048, 0.76161406],
       [0.46553375, 0.46553375, 0.48846686, 0.51184964, 0.53562709,
        0.55974008, 0.58412587, 0.60871871, 0.63345058, 0.65825186,
        0.68305217, 0.70778112, 0.73236915, 0.75674826, 0.7808528 ],
       [0.46231851, 0.46231851, 0.48618346, 0.51062704, 0.53559861,
        0.56104225, 0.58689721, 0.61309834, 0.63957679, 0.66626064,
        0.69307565, 0.71994615, 0.74679581, 0.77354854, 0.80012935],
       [0.45823267, 0.45823267, 0.48298553, 0.50845584, 0.53459977,
        0.56136718, 0.58870191, 0.61654211, 0.64482069, 0.67346592,
        0.70240208, 0.73155024, 0.76082911, 

In [14]:
Q = np.zeros((len(A),len(S)))

In [15]:
Q

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
def initQwithLargeConstantValue(Q): 
    for i in range (len(Q)):
        for j in range(len(Q[0])):
            Q[i][j]=2
    return Q

In [17]:
Q2 = initQwithLargeConstantValue(Q)

In [18]:
Q2

array([[2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       ...,
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.]])

In [19]:
#Créer une fonction qui trouve le maximum de Q(s,a)
def trouve_max(Q,s_prime):
    sol = Q[0][s_prime]
    arg = 0
    for i in range (1,len(Q)): 
        if sol <= Q[i][s_prime]:
            sol = Q[i][s_prime]
            arg = i
    return sol, arg        

In [20]:
trouve_max(Q,1)

(2.0, 14)

In [23]:
def updateQ(Q,A, S,a_t,s_t, alpha, delta, pi_t, s_t1):
    Qbis = Q
    for i in range(len(Q)): 
        for j in range(len(Q[0])):
            if i == a_t and j == s_t: #faire attention à différencier valeurs et indices : ici on parle d'indice
                Qbis[i][j]= (1-alpha)*Q[i][j] + alpha*(pi(s_t1) + delta*trouve_max(Q,s_t1)[0])
                a_t1 = trouve_max(Q,s_t1)[1]

    return Qbis, a_t1

In [24]:
updateQ(Q,A,S,1,0, 0.5, 0.15, 2)

TypeError: 'int' object is not subscriptable